In [29]:
from model import *
import numpy as np
from sklearn.svm import SVR
from utils import *
from time import time
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors

In [84]:
import numpy as np
from sklearn.svm import SVR
from utils import *
from scipy.spatial import distance

class S3VR():
    """
    The S3VR class used to do semi-surpervised regression tasks
    """
    def __init__(self, k_local, k_global, r, beta):
        """Initialization

        Args:
            k_local (int): k values for PLR_local
            k_global (int): k values for PLR_global
            r (float): probability threshold for choosing data pointsw
            beta (float): variance parameter
        """
        self.svr = None
        self.labeled_X = None
        self.unlabeled_X = None
        self.y = None
        self.dist = None

        self.k_local = k_local
        self.k_global = k_global
        self.r = r
        self.beta = beta

    def rbf(self, x1, x2, l=1):
        """Calulate the kernel mappings for x1 and x2

        Args:
            x1 (np.ndarray): the x1 with shape (n_1, d)
            x2 (np.ndarray): the x2 with shape (n_2, d)
            l (int, optional): the parameter of normal distribution. Defaults to 1.

        Returns:
            np.ndarray: the rbf kernel vectors
        """
        if ((x1-x2)**2).ndim > 1:
            return np.exp(-1 / (2 * (l**2)) * ((x1-x2)**2).sum(axis=1))
        else:
            return np.array([np.exp(-1 / (2 * (l**2)) * ((x1-x2)**2).sum())])

    def calc_distance(self):
        self.dist = euclidean_distances(self.unlabeled_X, self.labeled_X)
        self.dist = np.argsort(self.dist, axis=1)

    def find_nn(self, index, k):
        """Find the k nearest neighbors of the point

        Args:
            point (np.ndarray): the data point with shape (d,)
            k (int): the number of nearest neighbors

        Raises:
            ValueError: the labeled X is not loaded

        Returns:
            np.ndarray: the matrix of the nearest neighbors with shape (k, d)
        """
        if self.labeled_X is None:
            raise ValueError("Not load datasets")
        if self.dist is None:
            self.calc_distance()
        index = self.dist[index, :k]
        return self.labeled_X[index], index

    def estimate_distribution(self, is_local):
        """Estimate the distribution

        Args:
            is_local (bool): whether use k_local or k_global

        Raises:
            ValueError: the datasets are not loaded

        Returns:
            np.ndarray, np.ndarray, int: the y_bar with shape (n_unlabeled,), the sigma_2_hat with shape (n_unlabeled,), the number of data points
        """
        if (self.labeled_X is None) or (self.unlabeled_X is None) or (self.y is None):
            raise ValueError("Not load datasets")

        k = self.k_local if is_local else self.k_global

        ones = np.ones((k, 1))
        num = self.unlabeled_X.shape[0]
        y_hat = np.zeros((num, 1))
        sigma_2_hat = np.zeros((num, 1))

        for i in range(num):
            nn, nn_index = self.find_nn(i, k)
            k_star = self.rbf(self.unlabeled_X[i], nn).reshape(-1, 1)
            K_hat = self.rbf(nn, nn) - ones @ k_star.T - k_star @ ones.T + self.rbf(self.unlabeled_X[i], self.unlabeled_X[i]) * ones @ ones.T

            # Equation (15), PLR paper
            cov = self.beta * K_hat + np.identity(k)
            # Equation (14), PLR paper
            mu = cov @ (ones/k)

            y_bar_nn = self.y[nn_index].sum(axis=0) / k
            diff = self.y[nn_index].reshape(-1, 1) - y_bar_nn * ones

            y_hat[i] = y_bar_nn + mu.T @ diff
            sigma_2_hat[i] = diff.T @ cov @ diff / k

        return y_hat, sigma_2_hat, num

    def data_generation(self):
        """Generate the combined training datasets

        Raises:
            ValueError: the datasets are not loaded

        Returns:
            np.ndarray, np.ndarray: the combined training X with shape (n_combined, d) and y with shape (n_combined,)
        """
        if (self.labeled_X is None) or (self.unlabeled_X is None) or (self.y is None):
            raise ValueError("Not load datasets")

        self.t0 = time()
        # estimate distribution
        y_local, sigma_2_local, n = self.estimate_distribution(True)
        self.t1 = time()
        y_global, sigma_2_global, _ = self.estimate_distribution(False)
        self.t2 = time()
        
        # avoid dividing by zero
        sigma_2_local[sigma_2_local == 0] = 1e-20
        sigma_2_global[sigma_2_global == 0] = 1e-20

        # conjugate
        # Equation (11), S3VR paper
        y_bar_conjugate = (y_global/sigma_2_global + n*y_local/sigma_2_local) / (1/sigma_2_global + n/sigma_2_local)
        sigma_2_conjugate = 1 / (1/sigma_2_global + n/sigma_2_local)
        self.t3 = time()
        
        # generate
        max_sigma_2, min_sigma_2 = sigma_2_conjugate.max(), sigma_2_conjugate.min()
        # Equation (12), S3VR paper
        pu = (sigma_2_conjugate - min_sigma_2) / (max_sigma_2 - min_sigma_2)
        X_hat = np.vstack((self.labeled_X, self.unlabeled_X[(pu >= self.r).reshape(-1,)]))
        y_hat = np.append(self.y.copy(), y_bar_conjugate[pu >= self.r])
        self.t4 = time()

        return X_hat, y_hat

    def fit(self, labeled_X, y, unlabeled_X):
        """Train the S3VR model on the datasets

        Args:
            labeled_X (np.ndarray): the labeled X with shape (n_labeled, d)
            y (np.ndarray): the labels with shape (n_labeled,) corresponding to the labeled X
            unlabeled_X (np.ndarray): the unlabeled X with shape (n_unlabeled, d)
        """
        self.labeled_X = labeled_X
        self.y = y
        self.unlabeled_X = unlabeled_X

        X_hat, y_hat = self.data_generation()
        self.svr = SVR()
        self.svr.fit(X_hat, y_hat)
        self.t5 = time()

        print(f't1: {self.t1-self.t0}')
        print(f't2: {self.t2-self.t1}')
        print(f't2: {self.t3-self.t2}')
        print(f't2: {self.t4-self.t3}')
        print(f't2: {self.t5-self.t4}')

    def predict(self, X):
        """Predict on X using the trained S3VR model

        Args:
            X (np.ndarray): the datasets to be predicted with shape (n_X, d)

        Raises:
            ValueError: the svr is not trained

        Returns:
            np.ndarray: the predictions of the dataset X with shape (n_X,)
        """
        if not self.svr:
            raise ValueError("No SVR is fitted.")

        return self.svr.predict(X)

In [ ]:
s3vr = S3VR(5, 10, 0.5, 10)
s3vr.fit(labeled_X, y[:, 0], unlabeled_X)
preds = s3vr.predict(labeled_X)

In [ ]:
s3vr = S3VR(5, 10, 0.5, 10)
s3vr.fit(labeled_X, y[:, 0], unlabeled_X)
preds = s3vr.predict(labeled_X)

In [71]:
labeled_X = np.load('labeled_data.npy')
unlabeled_X = np.load('unlabeled_data.npy')
y = np.load('labels.npy')

In [ ]:
st = time()
distance.cdist(unlabeled_X, labeled_X, 'euclidean')
time() - st

In [ ]:
st = time()
for i in range(10):
    euclidean_distances(unlabeled_X, labeled_X)
time() - st

In [72]:
labeled_X = labeled_X.astype(np.float64)
unlabeled_X = unlabeled_X.astype(np.float64)
y = y.astype(np.float64)

In [ ]:
st = time()
for i in range(10):
    euclidean_distances(unlabeled_X, labeled_X)
time() - st

In [75]:
s3vr = S3VR(5, 10, 0.5, 10)
s3vr.fit(labeled_X, y[:, 0], unlabeled_X)
preds = s3vr.predict(labeled_X)

t1: 9.118608713150024
t2: 2.3686609268188477
t2: 0.0009975433349609375
t2: 0.0029926300048828125
t2: 2.6758415699005127


In [85]:
s3vr = S3VR(5, 10, 0.5, 10)
s3vr.fit(labeled_X, y[:, 0], unlabeled_X)
preds = s3vr.predict(labeled_X)

t1: 9.108633518218994
t2: 2.3606841564178467
t2: 0.0
t2: 0.0019953250885009766
t2: 2.8723158836364746
